# importing required libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
from tensorflow.keras

# Integer labels

# importing images from folder

In [2]:
base_dir = "/kaggle/input/brain-tumor/Brain-tumer" # Base path to data directory
categories = ['no_tumor', 'glioma_tumor', 'meningioma_tumor', 'pituitary_tumor'] # categories
label_map = {
    'no_tumor': 0,
    'glioma_tumor': 1,
    'meningioma_tumor': 2,
    'pituitary_tumor': 3
}# dictionary for category and corresponding integer

# Function for loading images

In [3]:
def load_images(base_dir,label_map):
    images=[]
    labels=[]
    for category in categories:
        category_path=os.path.join(base_dir,category)
        label=label_map[category]

        # listing every file in perticular directory
        for filename in os.listdir(category_path):
            # check file extension
            if (filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png")):
                image_path = os.path.join(category_path,filename)
                images.append(image_path)
                labels.append(label)
    return images,labels
        
images,labels=load_images(base_dir,label_map)

# Combine shuffle and seperate

In [4]:
combined = list(zip(images, labels))
np.random.shuffle(combined)
images, labels = zip(*combined)
print(images[0])

/kaggle/input/brain-tumor/Brain-tumer/glioma_tumor/glioma_tumor (2509).jpg


# Splitting data into train,validation and test

In [5]:
# 70% train, 30% temp
train_imgs, temp_imgs, train_labels, temp_labels = train_test_split(images, labels, test_size=0.3, stratify=labels)

# 50% of temp in each test and val
val_imgs, test_imgs, val_labels, test_labels = train_test_split(temp_imgs, temp_labels, test_size=0.5,stratify=temp_labels)

# Decoding and Preprocessing images 

In [6]:
new=tf.io.read_file(images[0])

In [7]:
new1=tf.io.decode_image(new)

In [8]:
new1.shape

TensorShape([512, 512, 1])

In [12]:
from PIL import Image
import numpy as np

def preprocess_images(image_paths, labels, batch_size=32):
    num_images = len(image_paths)
    processed_images = []
    processed_labels = []

    for start in range(0, num_images, batch_size):
        end = min(start + batch_size, num_images)

        batch_images = []
        batch_labels = []

        for img_path, label in zip(image_paths[start:end], labels[start:end]):
            # Open the image using Pillow
            image = Image.open(img_path).convert('RGB')  # Convert to RGB
            
            # Resize the image to 224x224
            image = image.resize((224, 224))
            
            # Convert the image to a numpy array
            image_array = np.array(image)
            
            # Normalize the image to the range [0, 1]
            image_array = image_array / 255.0
            
            # Add image array to the batch list
            batch_images.append(image_array)
            
            # Add label to the batch labels list
            batch_labels.append(label)

        # Convert batch lists to numpy arrays
        processed_images.append(np.array(batch_images, dtype=np.float32))
        processed_labels.append(np.array(batch_labels, dtype=np.int64))

    # Concatenate all processed batches into final arrays
    processed_images = np.concatenate(processed_images, axis=0)
    processed_labels = np.concatenate(processed_labels, axis=0)

    return processed_images, processed_labels

# Assuming train_imgs and train_labels are defined
train_images, train_labels = preprocess_images(train_imgs[:10000], train_labels[:10000], batch_size=2)
val_images, val_labels = preprocess_images(val_imgs[:10000], val_labels[:10000], batch_size=2)

print(train_images.shape)  # Should print something like (num_images, 224, 224, 3)
print(train_labels.shape)  # Should print (num_images,)




(10000, 224, 224, 3)
(10000,)


# Meta Model

In [14]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Dense , GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [15]:
from tensorflow.keras.optimizers import Adam

# Ensemble learning 

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense,Dropout
from tensorflow.keras.layers import Dense,Dropout, MaxPooling2D, Conv2D,BatchNormalization, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.models import load_model

# # Detect and initialize the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect the TPU hardware
# tf.tpu.experimental.initialize_tpu_system(tpu)  # Initialize the TPU system

# # Instantiate the TPU distribution strategy
# tpu_strategy = tf.distribute.TPUStrategy(tpu)

# #Define the model within the TPU strategy scope
# with tpu_strategy.scope():
        # Assuming you have a pre-trained base model (like MobileNet, EfficientNet, etc.)

# Loading both models
model_1=load_model("/kaggle/input/resnet50/tensorflow2/default/1/ResNet.keras")
model_2=load_model("/kaggle/input/custom1/tensorflow2/default/1/new_custom_model.keras")


#     for batch_images, batch_labels in train_data.take(1):
#         print("Image batch shape:", batch_images.shape)
#         print("Label batch shape:", batch_labels.shape)
#         break
#     print(train_images.shape)
# Assume X_val is your validation data
resnet_preds = model_1.predict(train_images,batch_size=8)  # ResNet model predictions
custom_preds = model_2.predict(train_images,batch_size=8)  # Custom model predictions

# Assume X_val is your validation data
resnet_preds_val = model_1.predict(val_images, batch_size=8)  # ResNet model predictions
custom_preds_val = model_2.predict(val_images, batch_size=8)  # Custom model predictions

# Stack predictions side by side
print(resnet_preds.shape)
print(custom_preds.shape)

combined_preds = np.column_stack((resnet_preds, custom_preds))
# Stack predictions side by side
combined_preds_val = np.column_stack((resnet_preds_val, custom_preds_val))


# Build a meta-model (simple fully connected network)
meta_model = Sequential([
Dense(128, activation='relu', input_shape=(combined_preds.shape[1],)),
Dense(64, activation='relu'),
Dense(4, activation='softmax')  # Assuming binary classification (adjust for multi-class)
])



optimizer=Adam(learning_rate=0.001)

early_stopping = EarlyStopping(
monitor='val_loss',
patience=3,  # Stop if no improvement for 3 epochs
verbose=1,
restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
'meta_model.keras',
monitor='val_loss',
save_best_only=True,
verbose=1
)

# Compile the model with optimizer, loss function, and metrics
meta_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model normally
meta_model.fit(combined_preds,train_labels,
          epochs=30, 
           validation_data=(combined_preds_val,val_labels),
           callbacks=[early_stopping,model_checkpoint]
#           steps_per_epoch=1500
         )


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 38s 28ms/step


I0000 00:00:1727592733.102973    1459 service.cc:145] XLA service 0x7e63e0002340 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727592733.103022    1459 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1727592733.103027    1459 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


  19/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step

I0000 00:00:1727592734.317032    1459 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step
647/647 ━━━━━━━━━━━━━━━━━━━━ 21s 32ms/step
647/647 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step
(10000, 4)
(10000, 4)
Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9422 - loss: 0.3877
Epoch 1: val_loss improved from inf to 0.04615, saving model to meta_model.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9423 - loss: 0.3869 - val_accuracy: 0.9876 - val_loss: 0.0461
Epoch 2/30
288/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9883 - loss: 0.0359
Epoch 2: val_loss improved from 0.04615 to 0.04337, saving model to meta_model.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9882 - loss: 0.0364 - val_accuracy: 0.9884 - val_loss: 0.0434
Epoch 3/30
293/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9859 - loss: 0.0445
Epoch 3: val_loss improved from 0.04337 to 0.04308, saving model to meta_model.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9860 - loss: 0.0444 - val_accuracy: 0.9878 - val_loss: 0.0431
Epoch 4/30
303/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9869 - loss: 0.0414
Epoch 4: val_loss did not improve from 0.04308
313/313 ━━━━━━━━━━

In [18]:
meta_model.save("meta_model.keras")

# Evaluation

In [19]:
meta_model.evaluate(test_data)